In [37]:
import pandas

import numpy
import pygsheets
import datetime
import re

In [49]:
excel_writer = pandas.ExcelWriter('AfricaGasTracker-Pipelines-SummarySheets-'+str(datetime.date.today())+'.xlsx')#, engine='xlsxwriter')


import Pipelines_Current dataset

In [50]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
#spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')
#spreadsheet = gc.open_by_key('1jP1GJbl-BjLbwb9BTblt0EQQr2hyMGNgdVL4I9_wQWQ') # January 2022 pipelines
spreadsheet = gc.open_by_key('1bfPrp0w8Ruorq08Qe4hD8M3xVJ5e00phZ6ApFivO98k') # December 2022 pipelines

gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A2')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A2')

#gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
#oil_pipes = oil_pipes.drop('WKTFormat', axis=1)
pipes_df_orig = pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#pipes_df_orig = gas_pipes.copy()

#get other relevant sheets
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()
owners_df_orig = spreadsheet.worksheet('title', 'Pipeline operators/owners (1/3)').get_as_df(start='A2')

# remove empty cells for pipes, owners
pipes_df_orig = pipes_df_orig[pipes_df_orig['PipelineName']!='']
owners_df_orig = owners_df_orig[owners_df_orig['Wiki']!='']
owners_df_orig.set_index('ProjectID', inplace=True)

In [51]:
country_ratios_df.replace('--', numpy.nan, inplace=True)

owners_df_orig.replace('',numpy.nan,inplace=True)
owners_df_orig.replace('--',numpy.nan,inplace=True)

pipes_df_orig.replace('--',numpy.nan,inplace=True)

In [52]:
region_df_orig = spreadsheet.worksheet('title', 'Region dictionary').get_as_df(start='A2')
region_df_africa = region_df_orig.loc[region_df_orig['AfricaGasTracker']=='Yes']
region_df_touse = region_df_africa.copy()

### create Africa-specific dataframes for region, country_ratios_df, owners_df

In [53]:
country_ratios_df_touse = country_ratios_df.loc[country_ratios_df['Country'].isin(
                                            region_df_touse['Country'].tolist())]

owners_df_touse = owners_df_orig.loc[
    ~owners_df_orig['Countries'].apply(
        lambda x: set(x.split(', ')).isdisjoint(set(region_df_touse['Country'].tolist()))
    )]

#owners_df_touse = owners_df_orig.loc[owners_df_orig['Countries'].str.contains(
#                                            '|'.join(region_df_touse['Country'].tolist()))]

pipes_df_touse = pipes_df_orig.loc[
    ~pipes_df_orig['Countries'].apply(
        lambda x: set(x.split(', ')).isdisjoint(set(region_df_touse['Country'].tolist()))
    )]

#pipes_df_orig.loc[pipes_df_orig['Countries'].str.contains(
#                                            '|'.join(region_df_touse['Country'].tolist()))]

### set up info to analyze Owners tab

In [54]:
# get a set list of all owners
owner_column_list = ['Owner1', 
                     'Owner2', 
                     'Owner3', 
                     'Owner4', 
                     'Owner5', 
                     'Owner6', 
                     'Owner7', 
                     'Owner8', 
                     'Owner9', 
                     'Owner10', 
                     'Owner11']

percent_column_list = ['Owner1%', 
                     'Owner2%', 
                     'Owner3%', 
                     'Owner4%', 
                     'Owner5%', 
                     'Owner6%', 
                     'Owner7%', 
                     'Owner8%', 
                     'Owner9%', 
                     'Owner10%', 
                     'Owner11%']

owner_id_list = ['ID1', 
                 'ID2', 
                 'ID3', 
                 'ID4', 
                 'ID5', 
                 'ID6', 
                 'ID7', 
                 'ID8', 
                 'ID9', 
                 'ID10', 
                 'ID11']

### sum MergedKmByCountry and MergedKmByRegion

In [55]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(set(region_df_touse.Country.to_list())) #sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(set(region_df_touse.Region.tolist())) #sorted(list(set(country_ratios_df_touse['Region'])))

In [56]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']

In [57]:
fuel_type = 'Gas'
#fuel_type = 'Oil'
#fuel_type = 'NGL'

country_ratios_df_subset = country_ratios_df_touse.copy()[country_ratios_df_touse['Fuel']==fuel_type]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby('Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby('Region')['MergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['In Development (Proposed + Construction)'] = km_by_region[['Proposed','Construction']].sum(axis=1)
km_by_country['In Development (Proposed + Construction)'] = km_by_country[['Proposed','Construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.name = 'Region'
km_by_country.index.name = 'Country'

totals_row = km_by_region.sum(axis=0)
totals_row.name = 'Total'
km_by_region = km_by_region.append(totals_row)

totals_row = km_by_country.sum(axis=0)
totals_row.name = 'Total'
km_by_country = km_by_country.append(totals_row)

km_by_region.to_excel(excel_writer, 'Kilometers by region')
km_by_country.to_excel(excel_writer, 'Kilometers by country')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_60655/907386446.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  km_by_region = km_by_region.append(totals_row)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_60655/907386446.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  km_by_country = km_by_country.append(totals_row)


In [58]:
km_by_country

,Proposed,Construction,In Development (Proposed + Construction),Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Country,,,,,,,,,
Algeria,1294.68,0.0,1294.68,133.09,314.00,13630.20,0.0,549.38,0.0
Angola,0.00,0.0,0.00,0.00,0.00,383.02,0.0,0.00,0.0
Benin,0.00,0.0,0.00,0.00,0.00,128.50,0.0,0.00,0.0
Botswana,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0
Burkina Faso,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0
Burundi,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0
Cabo Verde,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0
Cameroon,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0
Central African Republic,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0


In [59]:
owner_parent_calculations_df = pandas.DataFrame()

for idx,row in pipes_df_touse.iterrows():
    parent_string = row.Parent
    parent_list = re.sub(' \[.*?\]', '', parent_string).split('; ') # all entries must have a Owner [%] syntax
    percent_list = [float(i.rstrip('%'))/100. for i in re.findall('\\d+(?:\\.\\d+)?%', parent_string)]

    if parent_list.__len__()!=percent_list.__len__():
        if percent_list==[]:
            percent_list = [1/parent_list.__len__() for i in parent_list]
        else:
            nmissing = parent_list.__len__()-percent_list.__len__()
            # distribute nans evenly
            total = numpy.nansum(percent_list)
            leftover = 1-total
            percent_list += [leftover/nmissing]*nmissing
    for p_idx,parent in enumerate(parent_list):
        #print(percent_list[p_idx])
        #print(row.LengthMergedKm)
        owner_parent_calculations_df = pandas.concat([owner_parent_calculations_df, 
                                                      pandas.DataFrame([{'Parent':parent, 'ProjectID':row.ProjectID, 
                                                                         'FractionOwnership':percent_list[p_idx],
                                                                         'LengthMergedKm':row.LengthMergedKm}])])

owner_parent_calculations_df['KmOwnership'] = owner_parent_calculations_df['FractionOwnership']*owner_parent_calculations_df['LengthMergedKm']

In [60]:
unique_owner_list = owner_parent_calculations_df.Parent.sort_values().unique().tolist()

##################################################
# create km count by owner, status
##################################################
owners_km_by_status_df = pandas.DataFrame(0.0, index=unique_owner_list, columns=status_list)

for status in status_list:
    
    pipes_df_temporary = pipes_df_touse.loc[(pipes_df_touse.Status==status)&
                                           (pipes_df_touse.Fuel=='Gas')]
    op_temporary = owner_parent_calculations_df.loc[owner_parent_calculations_df.ProjectID.isin(pipes_df_temporary.ProjectID)]
    op_sum = pandas.DataFrame(op_temporary.groupby('Parent', dropna=False)['KmOwnership'].sum(min_count=0))
    owners_km_by_status_df.loc[:,status] = op_sum

owners_km_by_status_df.index.name = 'Parent Company'
owners_km_by_status_df['In Development (Proposed + Construction)'] = owners_km_by_status_df[['Proposed','Construction']].sum(axis=1, min_count=0)
owners_km_by_status_df = owners_km_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_km_by_status_df = owners_km_by_status_df[excel_status_list]

totals_row = owners_km_by_status_df.sum(axis=0)
totals_row.name = 'Total'
owners_km_by_status_df = owners_km_by_status_df.append(totals_row)

owners_km_by_status_df.replace(numpy.nan, 0, inplace=True)
owners_km_by_status_df = owners_km_by_status_df.loc[~(owners_km_by_status_df==0).all(axis=1)]

owners_km_by_status_df.to_excel(excel_writer, sheet_name='Pipeline km by company')

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_60655/1390775355.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  owners_km_by_status_df = owners_km_by_status_df.append(totals_row)


In [61]:
owners_km_by_status_df

,Proposed,Construction,In Development (Proposed + Construction),Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Parent Company,,,,,,,,,
APA Corporation,0.00,0.0,0.00,0.0000,0.00,369.500000,0.0,0.0,0.0
Agip,0.00,0.0,0.00,0.0000,0.00,143.000000,0.0,0.0,0.0
Atlas Petroleum International,0.00,0.0,0.00,0.0000,0.00,14.000000,0.0,0.0,0.0
Axxela,510.00,0.0,510.00,0.0000,0.00,0.000000,0.0,0.0,0.0
BASF,0.00,0.0,0.00,0.0000,0.00,129.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
TotalEnergies SE,0.00,0.0,0.00,11.9250,0.00,154.914313,0.0,0.0,0.0
Transnet,3949.19,0.0,3949.19,0.0000,0.00,600.000000,0.0,0.0,0.0
Tunisian Company of Electricity and Gas,0.00,0.0,0.00,0.0000,0.00,294.000000,0.0,0.0,0.0


### pipeline km by start year, type

In [62]:
pipes_started_eu = pipes_df_touse.copy()
#pipes_started_eu['StartYearLatest'].replace(numpy.nan,'',inplace=True)

pipes_started_eu = pipes_started_eu[(pipes_started_eu['Status'].isin(['Operating'])) &
                              (pipes_started_eu['Fuel']=='Gas')]
pipes_started_eu_sum = pipes_started_eu.groupby('StartYearEarliest')['LengthMergedKm'].sum()

In [63]:
km_by_start_year = pandas.DataFrame(index=list(range(1980,2023)), columns=['Gas pipeline km'])
km_by_start_year.index.name = 'Start year'

km_by_start_year['Gas pipeline km'] = pipes_started_eu_sum
km_by_start_year.replace(numpy.nan,0,inplace=True)

totals_row = km_by_start_year.sum(axis=0)
totals_row.name = 'Total'
km_by_start_year = km_by_start_year.append(totals_row)

km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_60655/2282834177.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  km_by_start_year = km_by_start_year.append(totals_row)


## save excel file

In [64]:
excel_writer.save()

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_60655/2638382022.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
